# News Headlines Sentiment Analysis

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [ ]:
# Read api key environment variable
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='3f7cf16e803f4ea3b86db8dea0597ef2')

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_news_en = newsapi.get_everything(
    q="bitcoin",
    language="en"
)

bitcoin_news_en["totalResults"]


In [ ]:
# Fetch the Ethereum news articles
eth_news_en = newsapi.get_everything(
    q="ethereum",
    language="en"
)

# Show the total number of news
eth_news_en["totalResults"]


In [ ]:
# Create the Bitcoin sentiment scores DataFrame
def create_df(news):
    articles = []
    for article in news['articles']:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": 'en'
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

    
btcdf = create_df(bitcoin_news_en)
btcdf

btcdf.head()

In [ ]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries
title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in btcdf.iterrows():
    try:
        # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["title"])
        title_sent["title_compound"].append(title_sentiment["compound"])
        title_sent["title_pos"].append(title_sentiment["pos"])
        title_sent["title_neu"].append(title_sentiment["neu"])
        title_sent["title_neg"].append(title_sentiment["neg"])
        title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
title_sentiment_df = pd.DataFrame(title_sent)
text_sentiment_df = pd.DataFrame(text_sent)
btcdf = btcdf.join(title_sentiment_df).join(text_sentiment_df)

btcdf.head()


In [ ]:
# Create the ethereum sentiment scores DataFrame
ethdf = create_df(eth_news_en)
ethdf

ethdf.head()

In [ ]:
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

# Sentiment scores dictionaries
title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in ethdf.iterrows():
    try:
        # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["title"])
        title_sent["title_compound"].append(title_sentiment["compound"])
        title_sent["title_pos"].append(title_sentiment["pos"])
        title_sent["title_neu"].append(title_sentiment["neu"])
        title_sent["title_neg"].append(title_sentiment["neg"])
        title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))

        text_sentiment = analyzer.polarity_scores(row["text"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
title_sentiment_df = pd.DataFrame(title_sent)
text_sentiment_df = pd.DataFrame(text_sent)
ethdf = ethdf.join(title_sentiment_df).join(text_sentiment_df)

ethdf.head()


In [ ]:
# Describe the Bitcoin Sentiment
btcdf.describe()

In [ ]:
# Describe the Ethereum Sentiment
ethdf.describe()

In [ ]:
btcdf.plot(
    y=["title_sent", "text_sent"],
    kind="bar",
    title="News Title and Text Sentiment Comparisson",
    figsize=(10, 8),
    grid=True,
)

In [ ]:
ethdf.plot(
    y=["title_sent", "text_sent"],
    kind="bar",
    title="News Title and Text Sentiment Comparisson",
    figsize=(10, 8),
    grid=True,
)

# Tokenizer

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Expand the default stopwords list if necessary
stopwordslist=stopwords.words('english')
#'...'.join(punctuation)

In [ ]:
bitcoin_news_en
btc_list = btcdf.values.tolist()
x=bitcoin_news_en['articles']

# Complete the tokenizer function
# = list(btcdf)

wordslist=[]
lemmatizer = WordNetLemmatizer()

def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    tokens = word_tokenize(text)

    # Convert the words to lowercase
    final_list=[]
    for i in range(0, len(tokens)):
        if tokens[i] not in punctuation and tokens[i] not in stopwordslist:
            final_list.append(tokens[i].lower())
            
    # Remove the punctuation
    
    # Remove the stop words
    
    # Lemmatize Words into root words
    lem = [lemmatizer.lemmatize(word) for word in final_list]

    return lem

#btcdf['tokens'] = ''

#for index,row in btcdf.iterrows():
    #btcdf.loc[ index, 'tokens'] = tokenizer(row['text'])
#    row['tokens']= tokenizer(row['text'])
    
btcdf.text = btcdf.text.astype('str')
y=tokenizer(btcdf.text.str.cat())





In [ ]:
# Create a new tokens column for bitcoin
btcdf['tokens'] = btcdf.text.apply(tokenizer)
btcdf.head(){


In [ ]:
# Create a new tokens column for ethereum
ethdf['tokens'] = ethdf.text.apply(tokenizer)
ethdf

---

# NGrams and Frequency Analysis

Look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
btcbigrams = ngrams(y, n=2)
btcbigrams

In [ ]:
# Generate the Ethereum N-grams where N=2
ethbigrams = ngrams(eth, n=2)
ethbigrams

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def bigram_counter(corpus): 
    # Combine all articles in corpus into one large string
    big_string = ' '.join(corpus)
    processed = process_text(big_string)
    bigrams = ngrams(processed, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['bigram', 'count'])



def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    bigrams = ngrams(tokens, n=2)
    top_10 = dict(Counter(bigrams).most_common(10))
    
    return Counter(tokens).most_common(N)

toptenbtc=token_count(y)

In [ ]:
# Get the top 10 words for Bitcoin
toptenbtc

In [ ]:
# Get the top 10 words for Ethereum




# Word Clouds

Generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
wc = WordCloud().generate(' '.join(y))
plt.imshow(wc)


In [ ]:
# Generate the Ethereum word cloud
wc = WordCloud().generate(eth)
plt.imshow(wc)


# Named Entity Recognition

Build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# download a language model for SpaCy
# !python -m spacy download en_core_web_sm




In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
x=' '.join(y)


In [ ]:
# Run the NER processor on all of the text
doc=nlp(x)
doc

# Add a title to the document
doc.user_data["title"] = "BITCOIN Rocks!!"

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
ents = list(doc.ents)
ents

---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
ethdf.text = ethdf.text.astype('str')
x=tokenizer(ethdf.text.str.cat())

eth=' '.join(x)


In [ ]:
# Run the NER processor on all of the text
doc=nlp(eth)
doc

# Add a title to the document
doc.user_data["title"] = "ETHEREUM Rocks!!"

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
ents = list(doc.ents)
ents